# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

F. Walter  ->  F. Walter  |  ['F. Walter']
S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']
Y. Wu  ->  Y. Wu  |  ['Y. Wu']
J. Matharu  ->  J. Matharu  |  ['J. Matharu']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']


K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']


G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
F. Walter  ->  F. Walter  |  ['F. Walter']
Arxiv has 72 new papers today
          11 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/11 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2511.02902
extracting tarball to tmp_2511.02902... done.


F. Walter  ->  F. Walter  |  ['F. Walter']
S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']


Found 142 bibliographic references in tmp_2511.02902/main_submit.bbl.
Retrieving document from  https://arxiv.org/e-print/2511.02905
extracting tarball to tmp_2511.02905... done.
Retrieving document from  https://arxiv.org/e-print/2511.02926


extracting tarball to tmp_2511.02926... done.


...

/opt/hostedtoolcache/Python/3.9.24/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure /comparison_ADI_CDI.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.24/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=0.615\linewidth]{/comparison_ADI_CDI.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.24/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure /throughput_ADI_CDI.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.24/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=0.33\linewidth]{/throughput_ADI_CDI.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


Issues with the citations
undefined string in line 377: procspie
Retrieving document from  https://arxiv.org/e-print/2511.03659


extracting tarball to tmp_2511.03659...

 done.


F. Walter  ->  F. Walter  |  ['F. Walter']


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.02902-b31b1b.svg)](https://arxiv.org/abs/2511.02902) | **A close look at the black hole masses and hot dusty toruses of the first quasars with MIRI-MRS**  |
|| S. E. I. Bosman, et al. -- incl., <mark>F. Walter</mark>, <mark>S. Belladitta</mark> |
|*Appeared on*| *2025-11-06*|
|*Comments*| *Submitted to ApJ; 26 pages, 12 figures*|
|**Abstract**|            The presence of supermassive black holes (SMBHs, $M_\text{BH}\sim10^9 M_\odot$) at $z>7$ remains a puzzle. While their existence appears to require exotic formation or growth processes, it is possible that BH mass estimates are incorrect due to differences from the low-$z$ quasars where BH mass scaling relations are calibrated. In this work, we employ JWST MIRI-MRS spectroscopy to measure the rest-frame optical/IR properties of the four highest-redshift known luminous type-1 quasars at $7.08\leq z<7.64$. We use three new broad lines to measure updated BH masses, H$\alpha$, Pa$\alpha$ and Pa$\beta$, finding them to be in the range $(4-15)\cdot10^8 M_\odot$. Our black hole mass estimates from all tracers agree with each other and with previous, less accurate, ground-based measurements based on MgII. The flux ratios of the H lines deviate from expectations for case A and B recombination in the same way as in $z<3$ quasars, indicating similar physical conditions in the Broad Line Region. Rest-frame near-IR continuum emission from a hot dusty torus surrounding the accretion disc is unambiguously detected in all four objects. We model the emission with SKIRTOR and constrain the inclination (face-on) and the opening angle ($\theta=40-60^\circ$) of the tori. These constraints are consistent for the four objects and with expectations from luminous quasars. We estimate a total dust mass $(1-4)\cdot10^6 M_\odot$ in the tori, corresponding to $(0.2-7)\%$ of the total dust in the quasar host galaxies. Given observed accretion rates, these SMBHs will deplete their tori in only $\sim5$ Myr. Overall, we confirm that $z>7$ SMBHs in quasars could not have grown from stellar-remnant BHs if the radiative efficiency of accretion is $10\%$. We also find no evidence that inferred BH masses and accretion processes in $z>7$ quasars differ significantly from their near-identical counterparts at $z<3$.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.02926-b31b1b.svg)](https://arxiv.org/abs/2511.02926) | **Euclid Quick Data Release (Q1): Hunting for luminous z > 6 galaxies in the Euclid Deep Fields -- forecasts and first bright detections**  |
|| E. Collaboration, et al. -- incl., <mark>J. Matharu</mark>, <mark>E. Bañados</mark>, <mark>S. Belladitta</mark>, <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-11-06*|
|*Comments*| **|
|**Abstract**|            The evolution of the rest-frame ultraviolet luminosity function (UV LF) is a powerful probe of early star formation and stellar mass build-up. At z > 6, its bright end (MUV < -21) remains poorly constrained due to the small volumes of existing near-infrared (NIR) space-based surveys. The Euclid Deep Fields (EDFs) will cover 53 deg^2 with NIR imaging down to 26.5 AB, increasing area by a factor of 100 over previous space-based surveys. They thus offer an unprecedented opportunity to select bright z > 6 Lyman break galaxies (LBGs) and constrain the UV LF's bright end. With NIR coverage extending to 2um, Euclid can detect galaxies out to z = 13. We present forecasts for the number densities of z > 6 galaxies expected in the final EDF dataset. Using synthetic photometry from spectral energy distribution (SED) templates of z = 5--15 galaxies, z = 1--4 interlopers, and Milky Way MLT dwarfs, we explore optimal selection methods for high-z LBGs. A combination of S/N cuts with SED fitting (from optical to MIR) yields the highest-fidelity sample, recovering >76% of input z > 6 LBGs while keeping low-z contamination <10%. This excludes instrumental artefacts, which will affect early Euclid releases. Auxiliary data are critical: optical imaging from the Hyper Suprime-Cam and Vera C. Rubin Observatory distinguishes genuine Lyman breaks, while Spitzer/IRAC data help recover z > 10 sources. Based on empirical double power-law LF models, we expect >100,000 LBGs at z = 6-12 and >100 at z > 12 in the final Euclid release. In contrast, steeper Schechter models predict no z > 12 detections. We also present two ultra-luminous (MUV < -23.5) candidates from the EDF-N Q1 dataset. If their redshifts are confirmed, their magnitudes support a DPL LF model at z > 9, highlighting Euclid's power to constrain the UV LF's bright end and identify the most luminous early galaxies for follow-up.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.02970-b31b1b.svg)](https://arxiv.org/abs/2511.02970) | **Euclid: Quick Data Release (Q1)- The connection between galaxy close encounters and radio activity**  |
|| M. Magliocchetti, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-11-06*|
|*Comments*| *22 pages, 16 figures, submitted to A&A*|
|**Abstract**|            Using the large statistics provided by both Euclid and the LOFAR surveys, we present the first large-scale study of the connection between radio emission, its morphology, and the merging properties of the hosts of radio sources up to z=2. By dividing the radio sample into active galactic nuclei (AGN) and star-forming galaxies, we find that radio-emitting AGN show a clear preference to reside within galaxies undergoing a merging event. This is more significant for AGN that present extended and/or complex radio emission: indeed, about half of them are associated with merging systems, while only 15% are hosted by an isolated galaxy. The observed trend is primarily driven by AGN residing at z < 1, especially in the case of high - P144MHz > 10^24 W Hz-1 sr-1 - radio luminosities (60% in mergers versus 10% isolated regardless of radio appearance). The situation is reversed in the case of radio-emitting star-forming galaxies, which are preferentially associated with isolated systems. This is more significant as we move towards low radio-luminosity/star-formation objects (P144MHz < 10^23 W Hz-1 sr-1) for which we find 40% in isolated systems versus 20% in mergers. These values hold regardless of redshift. We interpret the above result for AGN with their need to accrete outer gas from local encounters in order to trigger (radio) activity, especially in the case of extended radio emission such as hot-spots and lobes. This is mostly observed at z < 1, since in the local Universe galaxies are more gas deprived than their higher-redshift counterparts. Internal gas reservoirs instead seem sufficient to trigger star formation within the majority of galaxies, which indeed prefer to be associated with isolated systems at all redshifts probed. (abridged)         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.02988-b31b1b.svg)](https://arxiv.org/abs/2511.02988) | **Euclid: Quick Data Release (Q1) -- Secondary nuclei in early-type galaxies**  |
|| M. Fabricius, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-11-06*|
|*Comments*| *17 pages, 22 figures*|
|**Abstract**|            Massive early-type galaxies (ETGs) are believed to form primarily through mergers of less massive progenitors, leaving behind numerous traces of violent formation histories, such as stellar streams and shells. A particularly striking signature of these mergers is the formation of supermassive black hole (SMBH) binaries, which can create depleted stellar cores through interactions with stars on radial orbits - a process known as core scouring. The secondary SMBH in such systems may still carry a dense stellar envelope and thereby remain observable for some time as a secondary nucleus, while it is sinking towards the shared gravitational potential of the merged galaxy. We leverage Euclid's Q1 Early Release data to systematically search for secondary nuclei in ETGs. We present a preliminary sample of 666 candidate systems distributed over 504 hosts (some of which contain multiple secondary nuclei). The vast majority of these fall at separations of 3 kpc to 15 kpc, indicative of normal mergers. 44 fall at projected separations of less than 2 kpc. We argue those candidates at very close angular separations are unlikely to be a consequence of chance alignments. We show that their stellar masses are mostly too large for them to be globular clusters and that a significant subset are unresolved even at Euclid's spatial resolution, rendering them too small to be dwarf galaxies. These may represent the highest-density nuclei of a previously merged galaxy, currently sinking into the centre of the new, common gravitational potential and thus likely to host a secondary SMBH. We then demonstrate that convolutional neural networks offer a viable avenue to detect multiple nuclei in the thirty-times larger sky coverage of the future Euclid DR1. Finally, we argue that our method could detect the remnants of a recoil event from two merged SMBHs.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.03518-b31b1b.svg)](https://arxiv.org/abs/2511.03518) | **Coherent Differential Imaging of high-contrast extended sources with VLT/SPHERE**  |
|| A. Potier, et al. -- incl., <mark>G. Chauvin</mark> |
|*Appeared on*| *2025-11-06*|
|*Comments*| *Accepted in A&A*|
|**Abstract**|            High-contrast imaging relies on advanced coronagraphs and adaptive optics (AO) to attenuate the starlight. However, residual aberrations, especially non-common path aberrations between the AO channel and the coronagraph channel, limit the instrument performance. While post-processing techniques such as spectral or angular differential imaging (ADI) can partially address those issues, they suffer from self-subtraction and inefficiencies at small angular separations or when observations are conducted far from transit. We previously demonstrated the on-sky performance of coherent differential imaging (CDI), which offers a promising alternative. It allows for isolating coherent starlight residuals through speckle modulation, which can then be subtracted from the raw images during post-processing. This work aims to validate a CDI method on real science targets, demonstrating its effectiveness in imaging almost face-on circumstellar disks, which are typically challenging to retrieve with ADI. We temporally modulated the speckle field in VLT/SPHERE images, applying small phase offsets on the AO deformable mirror while observing stars surrounded by circumstellar material: HR 4796A, CPD-36 6759, HD 169142, and HD 163296. We hence separated the astrophysical scene from the stellar speckle field, whose lights are mutually incoherent. Combining a dozen of data frames and reference coronagraph point spread functions through a Karhunen-Loève image projection framework, we recover the circumstellar disks without the artifacts that are usually introduced by common post-processing algorithms (e.g., self-subtraction). The CDI method therefore represents a promising strategy for calibrating the effect of static and quasi-static aberrations in future direct imaging surveys. Indeed, it is efficient, does not require frequent telescope slewing, and does not introduce image artifacts to first order.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.03659-b31b1b.svg)](https://arxiv.org/abs/2511.03659) | **ALMA and JWST Imaging of $z\ >\ 6$ Quasars: No Spatial Position Offset Observed Between Quasars and Their Host Galaxies**  |
|| A. Wilde, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2025-11-06*|
|*Comments*| *Accepted to the Astrophysical Journal*|
|**Abstract**|            We present a study determining the spatial offset between the position of the supermassive black hole (as traced through their broad line regions) and the host galaxy in six $z > 6$ quasars. We determined the host galaxy's position from $\lesssim$$0.1^{\prime\prime}$ ($\lesssim$ 600 pc) resolution Atacama Large Millimeter/sub-millimeter Array (ALMA) [CII] 158 $\mu m$ and corresponding dust continuum imaging. We determined the quasar's position from $\lesssim$ 400 pc resolution James Webb Space Telescope Near-Infrared Camera (JWST NIRCam) imaging. We estimated the observational uncertainties on the quasar's position using astrometric data from the Global Astrometric Interferometer for Astrophysics (GAIA) of field stars within the NIRCam images. We find that all six quasars are found within the central $\sim 400$ pc of their host galaxy dust continuum and [CII] emission. Apparent offsets seen in rest-frame optical JWST observations are not detected in our ALMA data, suggesting they likely result from dust obscuration rather than a true physical separation between the SMBH and its host galaxy. Kinematic modeling of these data further reveals that none of the galaxies show evidence for recent merger activity, and most of the galaxies can be accurately modeled using a simple disk model. The lack of an offset supports theoretical models that predict that positional offset within these galaxies are either short-lived or intrinsically rare.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.02905-b31b1b.svg)](https://arxiv.org/abs/2511.02905) | **Dynamical evolution of stellar binaries in galactic centers**  |
|| M. Dodici, S. Tremaine, <mark>Y. Wu</mark> |
|*Appeared on*| *2025-11-06*|
|*Comments*| *16 pages, 10 figures. Submitted to ApJ. Comments welcome!*|
|**Abstract**|            Stellar binaries in galactic centers are relevant to several observable phenomena, including hypervelocity stars, X-ray binaries, and mergers of stars and compact objects; however, we know little about the properties of these binaries. Past works have suggested that a small fraction of them should contract to a few stellar radii or collide, due to the co-operation of stellar tides and the eccentricity oscillations induced by the strong tidal field of the central massive black hole. We revisit this model with several updates. We first argue that when a binary's pericenter separation is driven down to a few stellar radii, diffusive excitation of stellar tides should quickly contract the orbit, saving the stars from collision. Instead, the stars should end up as a very tight binary. We then show that vector resonant relaxation and perturbations from passing stars -- effects not included in past models -- dramatically increase the prevalence of such encounters. In numerical experiments, we find that 1 in 5 binaries around 1 pc from Sgr A* should tidally contract in this way while still on the main sequence. This rate climbs to 3 in 5 around 0.01 pc, inward of which it plateaus. We briefly discuss observable implications of these results, with particular attention to young stellar binaries in the Galactic Center.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.03281-b31b1b.svg)](https://arxiv.org/abs/2511.03281) | **A semi-analytical mock galaxy catalog for the CSST extragalactic surveys from the Jiutian simulations**  |
|| Z. Tan, et al. -- incl., <mark>L. Xie</mark> |
|*Appeared on*| *2025-11-06*|
|*Comments*| *accepted by SCPMA*|
|**Abstract**|            We introduce a mock galaxy catalog built for the CSST extragalactic surveys using the primary runs of the Jiutian $N$-body simulation suites. The catalogs are built by coupling the GAlaxy Evolution and Assembly (GAEA) semi-analytical model of galaxy formation with merger trees extracted from the simulations using the Hierarchical Bound-Tracing (HBT+) algorithm. The spectral energy distributions (SEDs) and broadband magnitudes are computed using the neural-network-based stellar population synthesizer StarDuster, which is trained on radiative transfer simulations to account for detailed galaxy geometry in modeling dust obscuration. Galaxy light-cones up to $z=5$ are subsequently generated with the BLiC light-cone builder which interpolates the properties of galaxies over time using an optimized interpolation scheme. The resulting catalogs exhibit good convergence in many statistical properties of the galaxy population produced from two different resolution simulations. The catalogs reproduce a number of observed galaxy properties across a range of galaxy mass and redshift, including the stellar mass functions, the luminosity function, gas mass fraction, galaxy size-mass relation and galaxy clustering. We also present the photometric and redshift distributions of galaxies expected to be observed in the CSST surveys.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2511.02902/./J0313_full_posterior.png', 'tmp_2511.02902/./J0313_paper.png', 'tmp_2511.02902/./J1342_paper.png', 'tmp_2511.02902/./J1007_paper.png', 'tmp_2511.02902/./J1120_paper.png', 'tmp_2511.02902/./SBS_1040+567.png']
copying  tmp_2511.02902/./J0313_full_posterior.png to _build/html/
copying  tmp_2511.02902/./J0313_paper.png to _build/html/
copying  tmp_2511.02902/./J1342_paper.png to _build/html/
copying  tmp_2511.02902/./J1007_paper.png to _build/html/
copying  tmp_2511.02902/./J1120_paper.png to _build/html/
copying  tmp_2511.02902/./SBS_1040+567.png to _build/html/
exported in  _build/html/2511.02902.md
    + _build/html/tmp_2511.02902/./J0313_full_posterior.png
    + _build/html/tmp_2511.02902/./J0313_paper.png
    + _build/html/tmp_2511.02902/./J1342_paper.png
    + _build/html/tmp_2511.02902/./J1007_paper.png
    + _build/html/tmp_2511.02902/./J1120_paper.png
    + _build/html/tmp_2511.02902/./SBS_1040+567.png
found figures ['tmp_2511.02926/./Plots/Confusi

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# A close look at the black hole masses and hot dusty toruses of the first quasars with MIRI-MRS

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.02902-b31b1b.svg)](https://arxiv.org/abs/2511.02902)<mark>Appeared on: 2025-11-06</mark> -  _Submitted to ApJ; 26 pages, 12 figures_

</div>
<div id="authors">

S. E.~I.~Bosman, et al. -- incl., <mark>F. Walter</mark>, <mark>S. Belladitta</mark>

</div>
<div id="abstract">

**Abstract:** The presence of supermassive black holes (SMBHs, $M_\text{BH} \sim 10^9 M_\odot$ ) at $z>7$ remains a puzzle.While their existence appears to require exotic formation or growth processes, it is possible that BH mass estimates are incorrect due to differences with respect to the low- $z$ quasars on which BH mass scaling relations are calibrated.In this work, we employ JWST MIRI-MRS spectroscopy to measure the rest-frame optical/IR properties of the four highest-redshift known luminous type-1 quasars at $7.08\leq z<7.64$ .We use three new broad lines to measure updated BH masses, H $\alpha$ , Pa $\alpha$ and Pa $\beta$ , finding them to be in the range $(4-15) \cdot 10^8 M_\odot$ . Our black hole mass estimates from all tracers agree with each other and with previous, less accurate, ground-based measurements based on Mg ${II}$ . The flux ratios of the H lines deviate from expectations for case A and B recombination in the same way as in $z<3$ quasars, indicating similar physical conditions in the Broad Line Region.Rest-frame near-IR continuum emission from a hot dusty torus surrounding the accretion disc is unambiguously detected in all four objects. We model the emission with SKIRTOR and constrain the inclination (face-on) and the opening angle ( $\theta=40-60^\circ$ ) of the tori. These constraints are consistent for the four objects and with expectations from luminous quasars. We estimate a total dust mass $(1-4) \cdot 10^6 M_\odot$ in the tori, corresponding to $(0.2-7)\%$ of the total dust in the quasar host galaxies. Given observed accretion rates, these SMBHs will deplete their tori in only $\sim5$ Myr.Overall, we confirm the fact that $z>7$ SMBHs in quasars could not have grown from stellar-remnant BHs if the radiative efficiency of accretion is $10\%$ . We also find no evidence that inferred BH masses and accretion processes in $z>7$ quasars differ significantly from their near-identical counterparts at $z<3$ .

</div>

<div id="div_fig1">

<img src="tmp_2511.02902/./J0313_full_posterior.png" alt="Fig9" width="100%"/>

**Figure 9. -** Posterior likelihood of torus parameters for quasar J0313$-$1806. The marginalised likelihoods show good constraining power on several parameters (opening angle $\theta$, inclination $i$, radial dust distribution index $p$) but not on others (polar dust distribution index $q$, optical depth $\tau_{9.7}$, radial extent $R$). The grey dashed lines and shaded regions show the parameter values with the highest marginalised likelihoods and $1\sigma$ confidence intervals. (*fig:posterior*)

</div>
<div id="div_fig2">

<img src="tmp_2511.02902/./J0313_paper.png" alt="Fig5.1" width="25%"/><img src="tmp_2511.02902/./J1342_paper.png" alt="Fig5.2" width="25%"/><img src="tmp_2511.02902/./J1007_paper.png" alt="Fig5.3" width="25%"/><img src="tmp_2511.02902/./J1120_paper.png" alt="Fig5.4" width="25%"/>

**Figure 5. -** MRS spectra of the four $z>7$ quasars in our sample; flux is in black and uncertainties are in grey. Channel 4 is not shown for J1342+0928 because the observations were badly affected by cosmic ray showers, prohibiting the extraction of a spectrum. Vertical lines denote the location of detected emission lines. (*fig:all*)

</div>
<div id="div_fig3">

<img src="tmp_2511.02902/./SBS_1040+567.png" alt="Fig2" width="100%"/>

**Figure 2. -** Photometry and spectroscopy of quasar SBS 1040+567 at $z=1.96$, compared with multiple models. Using the data in black, the best-fit power-law plus accretion disc fit return a very cold hot torus temperature of $T=983$ K  ([Hernán-Caballero, et. al 2016]()) . However, a direct rescaling of our SKIRTOR model for the $z=7.5$ quasar J0313$-$1806 (red) provides an equivalent or better fit, despite the fact the hot dust temperature we would measure for this fit in the corresponding wavelength range is $T=1440$ K. This temperature bias likely comes from the fact that the torus contains dust at multiple temperatures which dominate at different rest-frame wavelengths. The $\lambda_\text{rest}$ range covered by our MRS spectra is shaded in red. (*fig:lowz*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.02902"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$</div>



<div id="title">

# Euclid Quick Data Release (Q1): Hunting for luminous $z > 6$ galaxies in the Euclid Deep Fields -- forecasts and first bright detections

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.02926-b31b1b.svg)](https://arxiv.org/abs/2511.02926)<mark>Appeared on: 2025-11-06</mark> - 

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>J. Matharu</mark>, <mark>E. Bañados</mark>, <mark>S. Belladitta</mark>, <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The evolution of the rest-frame ultraviolet luminosity function (UV LF) is a powerful probe of early star formation and galaxy stellar mass build-up. At $z>6$ , its bright end ( $M_{\rm UV} <  -21$ ) remains poorly constrained due to small survey volumes of existing near-infrared (NIR) space-based imaging surveys. The Euclid Deep Fields (EDFs) will cover 53 deg $^2$ with NIR coverage down to 26.5 AB magnitude, providing a factor of 100 increase in area compared to previous space-based surveys. They thus offer an unprecedented opportunity to select bright $z>6$ Lyman break galaxies (LBGs) and definitively constrain the bright end of the UV LF. With its NIR coverage extending to $\sim2  \si{\micron}$ , $\Euclid$ has the power to detect galaxies out to $z\sim13$ . Here, we present a forecast for the number densities of $z>6$ galaxies that $\Euclid$ is expected to observe in the final EDF dataset. Using synthetic photometry from spectral energy distribution (SED) templates of $z=5$ --15 galaxies, $z=1$ --4 interlopers, and Milky Way MLT dwarfs, we investigate optimal selection methodologies for high- $z$ LBGs in the EDF datasets. We find that a combination of S/N cuts with SED fitting (over optical to MIR bands) yields the highest fidelity sample, recovering more than $76\%$ of the input synthetic $z>6$ LBGs, while limiting low- $z$ contamination to less than $10\%$ . This contamination does not include effects from instrumental artefacts, which will impact the first $\Euclid$ data releases. Auxiliary data proves critical: optical coverage from Hyper Suprime Camera and $_ Vera C. Rubin_$ Observatory will distinguish genuine Lyman breaks from contaminant features, while $\Spitzer$ /IRAC data is vital for recovering $z>10$ sources. Based on empirical double power-law LF models, we expect more than $100 000$ LBGs at $z=6$ --12 and more than $100$ sources as far back as $z>12$ in the final $\Euclid$ data release. In contrast, the steeper Schechter LF models predict no detections of $z>12$ LBGs. In this work, we also present two ultra-luminous ( $M_{\rm UV} < -23.5$ ) candidates selected from the Q1 EDF-N dataset (Euclid Quick Data Release). If their redshifts are reliable, their magnitudes suggest a DPL UV LF model at $z>9$ . This highlights the power of $\Euclid$ in constraining the bright end of the UV LF in the early Universe and in identifying the most luminous sources that are valuable for further follow-up observations.

</div>

<div id="div_fig1">

<img src="tmp_2511.02926/./Plots/ConfusionMatrices/SEP_vis-dropouts_confmatrices_v2025.png" alt="Fig8.1" width="33%"/><img src="tmp_2511.02926/./Plots/ConfusionMatrices/CDFS_vis-dropouts_confmatrices_v2025.png" alt="Fig8.2" width="33%"/><img src="tmp_2511.02926/./Plots/ConfusionMatrices/NEP_vis-dropouts_confmatrices_v2025.png" alt="Fig8.3" width="33%"/>

**Figure 8. -** Confusion matrices, for each EDF, presenting the performance of our classification (Sect. \ref{sc:EF_Paper_Method}). The left matrices are normalised by true class, and the right matrices are normalised by selected class. Diagonals present the completeness (_left_) and purity (_right_).  (*fig:recoved_sel_frac_allfields*)

</div>
<div id="div_fig2">

<img src="tmp_2511.02926/./Plots/Appendix_Plots/SEP_v_euclid-only_confmatrices.png" alt="Fig11.1" width="33%"/><img src="tmp_2511.02926/./Plots/Appendix_Plots/CDFS_v_euclid-only_confmatrices.png" alt="Fig11.2" width="33%"/><img src="tmp_2511.02926/./Plots/Appendix_Plots/NEP_v_euclid-only_confmatrices.png" alt="Fig11.3" width="33%"/>

**Figure 11. -** Performance of a classifier including \Euclid filters only, for each EDF. The performance of this classification is presented in confusion matrices, which are normalised by true class (_left_) and selected class (_right_). Diagonals present completeness (_left_) and purity (_right_). (*fig:recoved_sel_frac_euclidonly_allfields*)

</div>
<div id="div_fig3">

<img src="tmp_2511.02926/./Plots/Appendix_Plots/SEP_v_no-opt-ext_confmatrices.png" alt="Fig12.1" width="33%"/><img src="tmp_2511.02926/./Plots/Appendix_Plots/CDFS_v_no-opt-ext_confmatrices.png" alt="Fig12.2" width="33%"/><img src="tmp_2511.02926/./Plots/Appendix_Plots/NEP_v_no-opt-ext_confmatrices_Corr.png" alt="Fig12.3" width="33%"/>

**Figure 12. -** Performance of a classifier which includes both _ Spitzer_ and \Euclid filters, for each EDF. The performance of this classification is presented in confusion matrices, which are normalised by true class (left) and selected class (right). Diagonals present completeness (left) and purity (right). (*fig:recoved_sel_frac_nooptext_allfields*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.02926"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# $\Euclid$: Quick Data Release (Q1) -- The connection between galaxy close encounters and radio activity$\thanks{This paper is published on behalf of the Euclid Consortium.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.02970-b31b1b.svg)](https://arxiv.org/abs/2511.02970)<mark>Appeared on: 2025-11-06</mark> -  _22 pages, 16 figures, submitted to A&A_

</div>
<div id="authors">

M. Magliocchetti, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** Using the large statistics provided by both $\Euclid$ and the LOFAR surveys, we present the first large-scale studyof the connection between radio emission, its morphology, and the merging properties of the hosts of radio sources up to $z\sim 2$ . By dividing the radio sample into active galactic nuclei (AGN) and star-forming galaxies, we find that radio-emitting AGN show a clear preference to reside within galaxies undergoing a merging event. This is more significant for AGN that present extended and/or complex radio emission: indeed, about half of them are associated with merging systems, while only $\SI{\sim 15}\percent$ are hosted by an isolated galaxy. The observed trend is primarily driven by AGN residing at $z<1$ , especially in the case of high  -- $P_{144 \rm MHz}>10^{24}$ W  Hz $^{-1}$ sr $^{-1}$ -- radio luminosities ( $\SI{\sim 60}\percent$ in mergers versus $\SI{\sim 10}\percent$ isolated regardless of radio appearance). On the other hand, this preference seems to disappear at higher redshifts, where only bright AGN with extended radio emission still prefer galaxies undergoing a merging event. The situation is reversed in the case of radio-emitting star-forming galaxies, which are preferentially associated with isolated systems. This is more significant as we move towards low radio-luminosity/star-formation objects ( $P_{144  \rm MHz}<10^{23}$ W Hz $^{-1}$ sr $^{-1}$ ) for which we find $\SI{\sim 40}\percent$ in isolated systems versus $\SI{\sim 20}\percent$ in mergers. These values hold regardless of redshift. We interpret the above result for AGN with their need to accrete outer gas from local encounters in order to trigger (radio) activity, especially in the case of extended radio emission such as hot-spots and lobes. This is mostly observed at $z<1$ , since in the local Universe galaxies are more gas deprived than their higher-redshift counterparts. Internal gas reservoirs instead seem sufficient to trigger star formation within the majority of galaxies, which indeed prefer to be associated with isolated systems at all redshifts probed.

</div>

<div id="div_fig1">

<img src="tmp_2511.02970/./figures/AGN_mer_stats_zle1_RL_gehlerserrors.png" alt="Fig5.1" width="50%"/><img src="tmp_2511.02970/./figures/AGN_mer_stats_zge1le2_RL_gehlerserrors.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** Similar to Fig. 4, except that now different panels present different redshift/radio-luminosity (expressed in W Hz$^{-1}$ sr$^{-1}$ units) combinations as shown (see text for details). The point in the top-right part of the top-right panel is off the scale and refers to one single object hosted by a merging system.
 (*fig:hist_AGN_mer_z_P*)

</div>
<div id="div_fig2">

<img src="tmp_2511.02970/./figures/SF_mer_stats_zge05le1_RL.png" alt="Fig7.1" width="50%"/><img src="tmp_2511.02970/./figures/SF_mer_stats_zge1le2_RL.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** Similar to Fig. 6, except that now different panels present different redshift/radio-luminosity (expressed in W Hz$^{-1}$ sr$^{-1}$) combinations as shown (see text for details).
 (*fig:hist_SF_mer_z_P*)

</div>
<div id="div_fig3">

<img src="tmp_2511.02970/./figures/hist_z_zge05le2.png" alt="Fig1.1" width="33%"/><img src="tmp_2511.02970/./figures/hist_AGN_z_zge05le2.png" alt="Fig1.2" width="33%"/><img src="tmp_2511.02970/./figures/hist_SF_z_zge05le2.png" alt="Fig1.3" width="33%"/>

**Figure 1. -** Redshift distribution of LOFAR sources in the EDF-N region in the redshift interval $0.5< z < 2$. The left-hand panel refers to the whole sample, the middle panel to the sub-class of radio AGN, and the right-hand panel to star-forming galaxies. In each panel the solid line refers to the parent radio sample from the work of \citetalias{bisigello}, while the dashed histograms correspond to those sources with optical counterparts from the work of \citetalias{Q1-SP013}. The bottom panels show the percentages obtained from the ratios between the above quantities, together with the associated (Poissonian) errors. The middle panel additionally presents the trends for AGN with complex radio morphology, short-long dashed lines for the parent sample and dotted lines for the matched sample.
 (*fig:hist_z*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.02970"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\todo}[1]{\textcolor{red}{TODO: #1}}$
$\newcommand{\computed}[1]{\textcolor{black}{#1}}$
$\newcommand{\myfigurewidth}[0]{.9}$
$\newcommand{\linenumbers}[0]$
$\newcommand{\orcid}[1]$
$\newcommand{\numhavemultiple}[0]{\computed{129}}$
$\newcommand{\numone}[0]{\computed{316}}$
$\newcommand{\numtwo}[0]{\computed{86}}$
$\newcommand{\numthree}[0]{\computed{25}}$
$\newcommand{\morethanthree}[0]{\computed{18}}$
$\newcommand{\numcandhostswithphot}[0]{\computed{449}}$
$\newcommand{\numearlytypes}[0]{\computed{4420}}$
$\newcommand{\numinitialsampleobjects}[0]{\computed{15 172}}$
$\newcommand{\numetnodnuclei}[0]{\computed{3916}}$
$\newcommand{\numetwithdnuclei}[0]{\computed{504}}$
$\newcommand{\numcanddnuclei}[0]{\computed{666}}$
$\newcommand{\bwlmagtostmass}[0]{\computed{-0.50}}$
$\newcommand{\numcanddnucleiwithphotandz}[0]{\computed{648}}$
$\newcommand{\numcandlttenkpc}[0]{\computed{440}}$
$\newcommand{\numcandlttwokpc}[0]{\computed{44}}$
$\newcommand{\numcandlttenkpcwithspecz}[0]{\computed{120}}$
$\newcommand{\numcandlttwokpcwithspecz}[0]{\computed{14}}$
$\newcommand{\numltohpointeight}[0]{\computed{639}}$</div>



<div id="title">

# $\vspace{-0.25cm}$$\Euclid$: Quick Data Release (Q1) --Secondary nuclei in early-type galaxies$\thanks{This paper is published on behalf of the Euclid Consortium.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.02988-b31b1b.svg)](https://arxiv.org/abs/2511.02988)<mark>Appeared on: 2025-11-06</mark> -  _17 pages, 22 figures_

</div>
<div id="authors">

M. Fabricius, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** Massive early-type galaxies (ETGs; $M > 10^{11} M_\odot$ ) are believed to form primarily throughmergers of less massive progenitors, leaving behind numerous traces of violentformation histories, such as stellar streams and shells. A particularly strikingsignature of these mergers is the formation of supermassive black hole (SMBH)binaries, which can create depleted stellar cores through interactions withstars on radial orbits -- a process known as core scouring.The secondary SMBH in such systems may still carry a dense stellar envelope andthereby remain observable for some time as a secondary nucleus, while it is sinkingtowards the shared gravitational potential of the merged galaxy.Direct observations ofsecondary nuclei on sub-kiloparsec scales remain rare, with only a few notablecases, such as NGC 5419. Investigating such features and building up statisticsrequires both high spatial resolution and wide-field coverage, a capabilityuniquely provided by $\Euclid$ .In this study, we leverage $\Euclid$ ’s Q1 Early Release data to systematicallysearch for secondary nuclei in ETGs. We present a preliminary sample of $\numcanddnuclei$ candidate systems distributed over $\numetwithdnuclei$ hosts(some of which contain multiple secondary nuclei). The vast majority of thesefall at separations of $\SI{3}{\kilo\parsec}$ to $\SI{15}{\kilo\parsec}$ , indicativeof normal mergers. $\numcandlttwokpc$ fall at projected separations of less than $\SI{2}{\kilo\parsec}$ .In the discussion, we argue that this mostinteresting subset of secondary nucleus candidates -- those at very close angularseparations -- are unlikely to be a consequence of chance alignments. We show thattheir stellar masses are mostly too large for them to be globular clusters andthat a significant subset are unresolved even at $\Euclid$ 's spatial resolution,rendering them too small to be dwarf galaxies. These may representthe highest-density nuclei of a previously merged galaxy, currently sinking intothe centre of the new, common gravitational potential and thus likely to host asecondary SMBH. We then demonstrate that convolutional neural networks offer aviable avenue to detect multiple nuclei in the thirty-times larger sky coverageof the future $\Euclid$ DR1.Finally, we argue that our methodcould detect the remnants of a recoil event from two merged SMBHs: Two of oursecondary nuclei candidates are unresolved atthe $\Euclid$ spatial resolution, occurat projected physical separations of less than 2 kpc, and occur in hostsof $M > 10^{11} M_\odot$ , which makes them viable candidates.

</div>

<div id="div_fig1">

<img src="tmp_2511.02988/./figures/specz_photz.png" alt="Fig2" width="100%"/>

**Figure 2. -** Comparison of photometric redshift estimates by PHZ to
    spectroscopic measurements. We show all objects in our early-type subsample
    where spectroscopic data are available. The limited precision in the various
    space- and ground-based imaging channels still limits the accuracy of
    photometric redshifts in Q1. Any quantity that is derived from the
    photometric redshift thus needs to be taken with care.
      (*fig:specz_photz*)

</div>
<div id="div_fig2">

<img src="tmp_2511.02988/./figures/angular_separation.png" alt="Fig7" width="100%"/>

**Figure 7. -** Distribution of angular separations.
    We estimate angular separations of the secondary light peak centre
    from the centre of the host as determined by the Sérsic model fit.
    The centres of the candidate nuclei are estimated using a simple
    centre-of-mass approach within a manually placed aperture on the
    residual image. (*fig:angular_separation*)

</div>
<div id="div_fig3">

<img src="tmp_2511.02988/./figures/nuc_cand_observed_mags.png" alt="Fig9" width="100%"/>

**Figure 9. -** Distribution of observed magnitudes of the
    secondary nucleus candidates in our sample. No extinction correction
    has been applied here.
     (*fig:nuc_cand_observed_mags*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.02988"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]{\orcidlink{#1}}$
$\newcommand{\pz}{\phantom{0}}$
$\newcommand{\DS}[1]{{\textcolor{magenta}{DS: #1}}}$
$\newcommand{\Arefe}[1]{{\textcolor{red}{AA: #1}}}$</div>



<div id="title">

# Euclid Quick Data Release (Q1): The average far-infrared properties of $\Euclid$-selected star-forming galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.02989-b31b1b.svg)](https://arxiv.org/abs/2511.02989)<mark>Appeared on: 2025-11-06</mark> -  _Submitted to A&A as part of the second Euclid Q1 paper splash_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The first Euclid Quick Data Release (Q1) contains millions of galaxies with excellent optical and near-infrared (IR) coverage.To complement this dataset, we investigate the average far-IR properties of $\Euclid$ -selected main sequence (MS) galaxies using existing $\Herschel$ and SCUBA-2 data.We use 17.6 deg $^2$ (2.4 deg $^2$ ) of overlapping $\Herschel$ (SCUBA-2) data, containing 2.6 million (240 000) MS galaxies. We bin the $\Euclid$ catalogue by stellar mass and photometric redshift and perform a stacking analysis following ${\tt SimStack}$ , which takes into account galaxy clustering and bin-to-bin correlations.We detect stacked far-IR flux densities across a significant fraction of the bins. We fit modified blackbody spectral energy distributions in each bin and derive mean dust temperatures ( $T_{\rm d}$ ), dust masses ( $M_{\rm d}$ ), and star-formation rates (SFRs). We find similar mean SFRs compared to the $\Euclid$ catalogue, and we show that the average dust-to-stellar mass ratios decreased from $z {\simeq} 1$ to the present day. Average dust temperatures are largely independent of stellar mass and are well-described by the function $T_2 {+} (T_1 {-} T_2) {\rm e}^{-t/\tau}$ , where $t$ is the age of the Universe, $T_1 {=} (79.7\pm7.4) $ K, $T_2 {=} (23.2\pm0.1) $ K, and $\tau{=} (1.6\pm0.1) $ Gyr. We argue that since the dust temperatures are converging to a non-zero value below $z {=} 1$ , the dust is now primarily heated by the existing cooler and older stellar population, as opposed to hot young stars in star-forming regions at higher redshift. We show that since the dust temperatures are independent of stellar mass, the correlation between dust temperature and SFR depends on stellar mass. Lastly, we estimate the contribution of the $\Euclid$ catalogue to the cosmic IR background (CIB), finding that it accounts for ${>} 60\%$ of the CIB at 250, 350, and 500 $\mu$ m.As the $\Euclid$ mission progresses, larger catalogues will allow us to probe the far-IR properties of MS galaxies out to higher redshifts and lower stellar masses, potentially recovering the complete CIB.

</div>

<div id="div_fig1">

<img src="tmp_2511.02989/./figures/Td_vs_t.png" alt="Fig2" width="100%"/>

**Figure 2. -** Best-fit dust temperatures from our SED fitting, $T_{\rm d}$, as a function of time (bottom axis) and redshift (top axis), considering only the redshift and stellar mass bins that are ${>} 95\%$ complete. We show the dust temperature evolution for five different stellar mass bins, with the stellar mass values of the centres of the bins given in the legend. The solid curve is a fit to the simple form $T_2 {+} (T_1 {-} T_2) e^{-t/\tau}$, while the dotted line is the quadratic-in-redshift fit from [Koprowski, Wijesekera and Dunlop (2024)]() and the dashed line is the linear-in-redshift fit from [Schreiber, Elbaz and Pannella (2017)](). We also show published mean temperature estimates for star-forming galaxies at low redshift \citep[$0.01 {<} z {<} 0.05$;][]{lamperti2019} in blue. (*fig:Td_z*)

</div>
<div id="div_fig2">

<img src="tmp_2511.02989/./figures/edff_summary_fig.png" alt="Fig9" width="100%"/>

**Figure 9. -** _ Top:_\Herschel-SPIRE data covering the CDFS-SWIRE field (which overlaps with the EDF-F) at 250, 350, and 500 $\mu$m. The blue contour shows the mask applied to the SPIRE images in order to remove bad edge pixels. The grey contours show the corresponding \Euclid catalogue mask, where masked rectangles are the locations of bright stars in the field that contaminate source extraction. _ Bottom:_ Same as the top panel, but  showing the RMS of the \Herschel-SPIRE data.  Coordinates are conventional RA and Dec. (*fig:field_summary_edff*)

</div>
<div id="div_fig3">

<img src="tmp_2511.02989/./figures/euclid_best_SED_combine.png" alt="Fig11" width="100%"/>

**Figure 11. -** Modified blackbody SEDs (with $\beta=1.96$ and $\alpha=2.3$) fit to the stacked \Herschel and SCUBA-2 flux densities. The redshifts and stellar masses of each bin are indicated by the top and right axis labels, respectively. The best-fit parameters are given in Table \ref{table:bestfit_sed}. SEDs have only been fit to bins where all three SPIRE flux densities are detected with S/N$ {>} 3$, and at least one PACS flux density is detected with S/N$ {>} 3$; panels are blank otherwise. Bins that are ${>} 95\%$ complete in stellar mass are highlighted in blue. (*fig:stack_sed_fits*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.02989"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\linenumbers}[0]$
$\newcommand{\hg}{\ensuremath{\mathrm{H} \gamma}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\tbd}[1]{{\color{violet}#1}}$
$\newcommand{\urgent}[1]{{\color{red}#1}}$
$\newcommand$
$\newcommand$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# Euclid Quick Data Release (Q1): Spectroscopic unveiling of highly ionised lines at $z=2.48$--3.88$\thanks{Dedicated to the memory of Olivier Le F{è}vre and Bianca Garilli, whose invaluable mentorship and guidance were instrumental to this work.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.03025-b31b1b.svg)](https://arxiv.org/abs/2511.03025)<mark>Appeared on: 2025-11-06</mark> - 

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** This study explores a rare population of sources in a currently uncharted region of spectroscopic redshift space in the Euclid Quick Data Release (Q1), and is intended potentially to support upcoming spectroscopic studies.Our goal is to identify and investigate a population of sources characterised by highly ionised emission lines in their spectra, which are indicative of active galactic nucleus activity, extreme shock phenomena, or Wolf--Rayet stars.A comprehensive visual inspection of spectra is conducted to ensure the reliability of the sample, focusing on the simultaneous detection of both $\NeV$ and $\OII$ emission-line measurements, a condition that restricts the $\Euclid$ spectroscopic redshift range to $z=2.48$ --3.88.To characterise this population, we analysed the morpho-spectrophotometric properties of their host galaxies. This allowed for a direct comparison with control sources that exhibit similar $\OII$ properties and spectroscopic redshifts, but not $\NeV$ lines.We identify sources solely based on spectroscopic criteria in the redshift range beyond the $\halpha$ regime.Encompassing 65 potential $\NeV$ candidates, the resulting sample delivers the first systematic probe of these $\NeV$ candidate emitters at high redshift.We found a good agreement, within 1 $\sigma$ , between the spectral measurements calculated using both direct integration and Gaussian fitting methodologies. The $\NeV$ candidates exhibit colours similar to bright QSOs,with only a few in the tail of very red quasars.We observed a higher stellar mass content, a lower continuum around the 4000 Å break, and a similar Sérsic index distribution compared to the control sample.This unique sample paves the way for a wide range of scientific investigations, which will be pursued in the forthcoming data releases.

</div>

<div id="div_fig1">

<img src="tmp_2511.03025/./figure1.png" alt="Fig1" width="100%"/>

**Figure 1. -** Comparison of the fluxes measured via the Gaussian fit (GF, $x$-axis) and the direct integration (DI, $y$-axis) for the \NeV and \OII emission lines used in this analysis (squares and circles, respectively). QSO- and galaxy-classified \NeV emitters are plotted with red and blue symbols, respectively. The dashed line represents the $x=y$ relation. The bands represent the $\pm$1 $\sigma$ distributions of each category. (*fig:gfdi*)

</div>
<div id="div_fig2">

<img src="tmp_2511.03025/./figure2a.png" alt="Fig2.1" width="50%"/><img src="tmp_2511.03025/./figure2b.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Distributions of the median stellar mass and the Sérsic index for the objects in the target sample (red) and control sample (grey). The dashed lines indicate the median value, while the vertical dotted lines and shaded bands represent the interquartile ranges(IQRs) of the distributions.
        The error bars represent the maximum dispersion in the parameters estimated via the bootstrap resampling and the IQR. The hatched bars represent the sources excluded from the analysis. (*fig:enia*)

</div>
<div id="div_fig3">

<img src="tmp_2511.03025/./figure3a.png" alt="Fig3.1" width="50%"/><img src="tmp_2511.03025/./figure3b.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** _ Top_: $(\IE - \YE)$ versus $(\JE - \HE)$ diagram showing the  \NeV galaxy-classified targets and \NeV QSO-classified target sources  (in blue and red circles) plotted with the \Euclid spectroscopically identified quasars in F25 (green). We overplot the region proposed in \citetalias{Q1-SP027} that identifies the QSO candidates (black-solid line), with the stellar locus found below this boundary at approximately \JE$-$\HE$< 0.1$ represented by purple contours in their Fig. 6. The grey points are the compact \Euclid sources in \citetalias{Q1-SP027}. _  Bottom_: $(\YE - \HE)$ versus $(\JE - \HE)$ diagram with the region proposed in \citetalias{Q1-SP023}  that identifies the red QSO (black-dashed line). Symbols are as in the upper panel. (*fig:jhiy*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.03025"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\sersic}{Sérsic }$
$\newcommand{\lb}[1]{\textcolor{red}{LB: #1}}$
$\newcommand{\linenumbers}[0]$
$\newcommand{\orcid}[1]$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# Euclid Quick Data Release (Q1): Searching for giant gravitational arcs in galaxy clusters with mask region-based convolutional neural networks

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.03064-b31b1b.svg)](https://arxiv.org/abs/2511.03064)<mark>Appeared on: 2025-11-06</mark> -  _12 pages, 6 figures_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** Strong gravitational lensing (SL) by galaxy clusters is a powerful probe of their inner mass distribution and a key test bed for cosmological models. However, the detection of SL events in wide-field surveys such as $\Euclid$ requires robust, automated methods capable of handling the immense data volume generated. In this work, we present an advanced deep learning (DL) framework based on mask region-based convolutional neural networks (Mask R-CNNs), designed to autonomously detect and segment bright, strongly-lensed arcs in $\Euclid$ 's multi-band imaging of galaxy clusters. The model is trained on a realistic simulated data set of cluster-scale SL events, constructed by injecting mock background sources into Euclidised $*Hubble*$ Space Telescope images of $10$ massive lensing clusters, exploiting their high-precision mass models constructed with extensive spectroscopic data. The network is trained and validated on over $4500$ simulated images, and tested on an independent set of $500$ simulations, as well as real $\Euclid$ Quick Data Release (Q1) observations. The trained network achieves high performance in identifying gravitational arcs in the test set, with a precision and recall of $76\%$ and $58\%$ , respectively, processing $2' \times 2'$ images in a fraction of a second. When applied to a sample of visually confirmed $\Euclid$ Q1 cluster-scale lenses, our model recovers $\approx 66\%$ of gravitational arcs above the area threshold used during training.  While the model shows promising results, limitations include the production of some false positives and challenges in detecting smaller, fainter arcs. Our results demonstrate the potential of advanced DL computer vision techniques for efficient and scalable arc detection, enabling the automated analysis of SL systems in current and future wide-field surveys. The code, \texttt{ARTEMIDE} , is open source and will be available at $\url{github.com/LBasz/ARTEMIDE}$ .

</div>

<div id="div_fig1">

<img src="tmp_2511.03064/fig/PAPER_HST2EUCLID_A1063_sersic_0_000000.png" alt="Fig4" width="100%"/>

**Figure 4. -** Steps of a \Euclid GCSL simulation. *Left*: \texttt{HST2EUCLID}$2'\times2'$ RGB image of the galaxy cluster Abell S1063 ($z_\mathrm{cl}=0.348$), with the main critical line (in red) for a source at $z_{\mathrm{s}}=2.92$, based on the lens model by \citet{bergamini2019}. The critical line has a circularised Einstein radius of $\theta_\mathrm{E} \simeq 33"$. The green cross marks the position of the injected source to be lensed. *Middle*: Source plane at $z_{\mathrm{s}}=2.92$, displaying the caustic (in red) related to the main critical line. The injected source (green cross) features a $\sersic$ profile (index $n=1.22$, $r_{\mathrm{eff}}=\ang{;;0.11}$), with $\YE=24.8$ and the SED of a star-forming galaxy; these parameters are sampled according to the procedure described in Sect. \ref{subsec:simulations}. *Right*: Colour-composite image of the simulated GCSL system, including the critical line (red dotted line). Green boxes enclose the gravitational arcs resulting from the lensing simulation, which are also shown in the bottom inset. (*fig:HST2EUCLID_A1063_sersic_0_000009*)

</div>
<div id="div_fig2">

<img src="tmp_2511.03064/fig/0.0.29-12-plot_precision_recall.png" alt="Fig6.1" width="33%"/><img src="tmp_2511.03064/fig/0.0.29-12-plot_precision_recall_score.png" alt="Fig6.2" width="33%"/><img src="tmp_2511.03064/fig/0.0.29-12-plot_precision_recall_threshold.png" alt="Fig6.3" width="33%"/>

**Figure 6. -** In panel (a) we show two sets of three $P$--$R$ curves for $\mathrm{IoU}_{\mathrm{thr}} = 0.5, 0.75$, @$50$:$5$:$95$, each obtained by varying the object score threshold $p_{\mathrm{thr}}$. In panel (b) we show the $P$--$R$ curve for $\mathrm{IoU}_{\mathrm{thr}} = 0.5$, where the dots are colour-coded by the score threshold $p_{\mathrm{thr}}$ used to evaluate them. In particular, we emphasise the $P$--$R$ pair associated to our choice of score threshold $p_{\mathrm{thr}}=0.996$. Note that the recall remains relatively high even at moderate precision, reflecting the network's ability to recover most bright arcs while keeping the number of false positives manageable. Increasing the threshold shifts the operating point toward higher precision at the expense of recall. In panel (c) we show two precision and the recall curves as a function of the score threshold for $\mathrm{IoU}_{\mathrm{thr}} = 0.5$. (*fig:pr-curves*)

</div>
<div id="div_fig3">

<img src="tmp_2511.03064/fig/plot_3_0.0.29.png" alt="Fig5" width="100%"/>

**Figure 5. -** *Left*: Euclidised $2' \times 2'$ RGB image ($R=JH_\sfont{E}$, $G=\YE$, $B=\IE$) of the galaxy cluster Abell 1063, belonging to the test set; the right-most arc is the one injected via SL simulation, while the others are real. *Right*: single channel $2' \times 2'$ Euclidised $\IE$ image of the same cluster. The green boxes enclose the gravitational arcs (both real and simulated) present in the field, i.e. the ground truth, while the red dashed boxes are the `gravitational arcs' found by the NN, having an object confidence score greater than $0.996$. (*fig:plot_3_0.0.29*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.03064"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\todo}[1]{\textcolor{red}{[#1]}}$
$\newcommand{\rga}[1]{\textcolor{magenta}{[#1]}}$
$\newcommand{\pba}[1]{\textcolor{blue}{[#1]}}$
$\newcommand{\froc}[2]{{#1}/{#2}}$
$\newcommand{\ra}[1]{\newcommand{\arraystretch}{#1}}$
$\newcommand{\arraystretch}{#1}$
$\newcommand{\Var}{{\textrm{Var}} }$</div>



<div id="title">

# Coherent differential imaging of high-contrast extended sources with VLT/SPHERE$\thanks{Based on observations collected at the European Southern Observatory, Chile, 113.26G2}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.03518-b31b1b.svg)](https://arxiv.org/abs/2511.03518)<mark>Appeared on: 2025-11-06</mark> -  _Accepted in A&A_

</div>
<div id="authors">

A. Potier, et al. -- incl., <mark>G. Chauvin</mark>

</div>
<div id="abstract">

**Abstract:** High-contrast imaging relies on advanced coronagraphs and adaptive optics (AO) to attenuate the starlight. However, residual aberrations, especially non-common path aberrations between the AO channel and the coronagraph channel, limit the instrument performance. While post-processing techniques such as spectral or angular differential imaging (ADI) can partially address those issues, they suffer from self-subtraction and inefficiencies at small angular separations or when observations are conducted far from transit. We previously demonstrated the on-sky performance of coherent differential imaging (CDI), which offers a promising alternative. It allows for isolating coherent starlight residuals through speckle modulation, which can then be subtracted from the raw images during post-processing. This work aims to validate a CDI method on real science targets using VLT/SPHERE, demonstrating its effectiveness in imaging almost face-on circumstellar disks, which are typically challenging to retrieve with ADI. We temporally modulated the speckle field in VLT/SPHERE images, applying small phase offsets on the AO deformable mirror while observing stars surrounded by circumstellar material: HR 4796A, CPD-36 6759, HD 169142, and HD 163296. We hence separated the astrophysical scene from the stellar speckle field, whose lights are mutually incoherent. Combining a dozen of data frames and reference coronagraph point spread functions through a Karhunen–Loève image projection framework, we recover the circumstellar disks without the artifacts that are usually introduced by common post-processing algorithms (e.g., self-subtraction). The CDI method therefore represents a promising strategy for calibrating the effect of static and quasi-static aberrations in future direct imaging surveys. Indeed, it is efficient, does not require frequent telescope slewing, and does not introduce image artifacts to first order.

</div>

<div id="div_fig1">

<img src="tmp_2511.03518/./1iter_example.png" alt="Fig8" width="100%"/>

**Figure 8. -** Image decomposition of HD 163296 through CDI. Left: Coronagraph image with no probe (i.e., total intensity image). Center: PWP-estimated speckle field. Right: CDI result (estimated astrophysical scene subtracted from the total intensity image) after one sequence of PWP. The negative values, indicated by black-colored regions, show local over-subtraction in areas where PWP cannot properly estimate the speckle field. (*fig:1iter_example*)

</div>
<div id="div_fig2">

<img src="" alt="Fig9.1" width="50%"/><img src="" alt="Fig9.2" width="50%"/>

**Figure 9. -** Comparison of disk images obtained with ADI (left column) and CDI (center column). For each disk, the ADI and CDI images are displayed using the same color scale; however, the scales vary from one disk to another. ADI and CDI results were obtained through the same dataset. We use K-klip ADI \citep{Soummer2012} with $K_{klip} = N_{filt}/2$. Right column: Comparison of disk intensities, for pixels inside the region encircled in green, obtained with both ADI and CDI as a function of the position angle. (*fig:Images_disk*)

</div>
<div id="div_fig3">

<img src="tmp_2511.03518/./Comparison_algorithms.png" alt="Fig1" width="100%"/>

**Figure 1. -** $I_{CDI}$ results for different reference estimations. Top left: $I_{ref,i}=0$, no CDI post-processing applied (equivalent to noADI). Top right: $I_{ref,i} = I_{PWP,i}$, a simple batch process used. Bottom left: Karhunen-Loève coherent component projection. Bottom right: Karhunen-Loève image projection. All images have been high-passed filtered, as explained in Sec. \ref{subsec:highpass_filter}. (*fig:Comparison_algorithms*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.03518"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# ALMA and JWST Imaging of $z > 6$ Quasars: No Spatial Position Offset Observed Between Quasars and Their Host Galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.03659-b31b1b.svg)](https://arxiv.org/abs/2511.03659)<mark>Appeared on: 2025-11-06</mark> -  _Accepted to the Astrophysical Journal_

</div>
<div id="authors">

A. Wilde, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We present a study determining the spatial offset between the position of the supermassive black hole (as traced through their broad line regions) and the host galaxy in six $z > 6$ quasars. We determined the host galaxy's position from $\lesssim0\farcs10$ ( $\lesssim$ 600 pc) resolution Atacama Large Millimeter/sub-millimeter Array (ALMA) $\CII$ 158 $\mu m$ and corresponding dust continuum imaging. We determined the quasar's position from $\lesssim$ 400 pc resolution James Webb Space Telescope Near-Infrared Camera (JWST NIRCam) imaging. We estimated the observational uncertainties on the quasar's position using astrometric data from the Global Astrometric Interferometer for Astrophysics (GAIA) of field stars within the NIRCam images. We find that all six quasars are found within the central $\sim 400$ pc of their host galaxy dust continuum and $\CII$ emission. Apparent offsets seen in rest-frame optical JWST observations are not detected in our ALMA data, suggesting they likely result from dust obscuration rather than a true physical separation between the SMBH and its host galaxy. Kinematic modeling of these data further reveals that none of the galaxies show evidence for recent merger activity, and most of the galaxies can be accurately modeled using a simple disk model. The lack of an offset supports theoretical models that predict that positional offset within these galaxies are either short-lived or intrinsically rare.

</div>

<div id="div_fig1">

<img src="tmp_2511.03659/./cont_mom_2054.png" alt="Fig5" width="100%"/>

**Figure 5. -** J2054$-$0005 continuum, \CII velocity-integrated flux density, mean velocity, and velocity dispersion maps. The cross marks the position of the quasar within the galaxy with error bars as found by comparing JWST and GAIA data (detailed in Section \ref{subsec:astrometry}). The contours in the top two panels start at 2$\sigma$ and increase by powers of $\sqrt{2}$. The synthesized beam is shown in the bottom left corner of each panel. (*fig:4pan 2054*)

</div>
<div id="div_fig2">

<img src="tmp_2511.03659/./cont_6pan_small_crimson_cropped.png" alt="Fig6" width="100%"/>

**Figure 6. -** Continuum imaging of all six sources in the sample. Previous optical position of the quasars are noted in red. JWST positions (see Section \ref{sec:jwst}) of the quasars are shown in white. For sources lacking reported uncertainties in the literature for their previous coordinates, the positions are marked with a dot. We note that the spatial scaling for J0109$-$3047 is different, for the clearest possible view of the galaxy. Contours and annotations are similar to Figure \ref{fig:4pan 2054}. (*fig:cont 6pan*)

</div>
<div id="div_fig3">

<img src="tmp_2511.03659/./2054_chanmaps_shifted.png" alt="Fig8" width="100%"/>

**Figure 8. -** J2054$-$0005 Channel Maps from the _qubefit_ thin disk model. The left panel shows \CII data at each 30 \kms increment with contours of the model. The right panel shows \CII emission at each 30 \kms increment with residual contours. Both sets of contours start at $2 \sigma$ and increase by factors of $\sqrt{2}$. (*fig:chanmap 2054*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.03659"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

122  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

21  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

9  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
